<a href="https://colab.research.google.com/github/crodier1/Age_Calculator/blob/master/Dow_Jones_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
dataset = pd.read_csv('dow_jones.csv')

dataset.drop('stock', inplace=True, axis=1)

In [ ]:
dates = pd.to_datetime(dataset['date']) 

dataset['weekday'] = pd.DatetimeIndex(dates).weekday
dataset['day'] = pd.DatetimeIndex(dates).day
dataset['month'] = pd.DatetimeIndex(dates).month
dataset['year'] = pd.DatetimeIndex(dates).year

dataset.drop('date', inplace=True, axis=1)


In [ ]:
def remove_dollar_symbol(amount):
  if amount == None:
    return
  return amount.replace('$', '')

In [ ]:
has_dollar_symbols = ['open', 'high', 'low', 'close', 'next_weeks_open', 'next_weeks_close']

for i in has_dollar_symbols:
  dataset[i] = dataset[i].apply(lambda num: remove_dollar_symbol(num))
  dataset[i] = dataset[i].astype(float)
  dataset[i] = dataset[i].replace('NaN', np.nan)
  dataset[i] = dataset[i].fillna(dataset[i].mean())

In [ ]:
for i in dataset.columns:   
  if dataset[i].isnull().sum() > 0:     
    dataset[i] = dataset[i].astype(float)    
    dataset[i] = dataset[i].fillna(dataset[i].mean())

In [ ]:
def get_dummies(colName):

  dummies = pd.get_dummies(dataset[colName])

  cols = dummies.columns

  dataset[[f"{colName}-{n}" for n in cols]] = dummies[cols]

  dataset.drop(colName, axis=1, inplace=True)

In [ ]:
for c in ['quarter', 'weekday', 'day', 'month', 'year']:
  get_dummies(c)


In [ ]:
y = dataset.percent_return_next_dividend.astype(int)

x = dataset.drop(['percent_return_next_dividend', 'year-2011'], axis=1)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train) 

In [ ]:
from sklearn.linear_model import PassiveAggressiveClassifier
clf = PassiveAggressiveClassifier(max_iter=1000, random_state=0,tol=1e-3)
clf.fit(X_train_scaled, y_train)
clf.score(X_test, y_test)

/usr/local/lib/python3.8/dist-packages/sklearn/base.py:443: UserWarning: X has feature names, but PassiveAggressiveClassifier was fitted without feature names
  warnings.warn(


0.8669354838709677

In [ ]:
from sklearn.linear_model import Perceptron
clf = Perceptron(tol=1e-3, random_state=0)
clf.fit(X_train_scaled, y_train)
clf.score(X_test, y_test)

/usr/local/lib/python3.8/dist-packages/sklearn/base.py:443: UserWarning: X has feature names, but Perceptron was fitted without feature names
  warnings.warn(


0.8669354838709677

In [ ]:
from sklearn.linear_model import RidgeClassifier
clf = RidgeClassifier().fit(X_train_scaled, y_train)
clf.score(X_test, y_test)

/usr/local/lib/python3.8/dist-packages/sklearn/base.py:443: UserWarning: X has feature names, but RidgeClassifier was fitted without feature names
  warnings.warn(


0.8669354838709677

In [ ]:
from sklearn.linear_model import RidgeClassifierCV
clf = RidgeClassifierCV(alphas=[1e-3, 1e-2, 1e-1, 1]).fit(X_train_scaled, y_train)
clf.score(X_test, y_test)

/usr/local/lib/python3.8/dist-packages/sklearn/base.py:443: UserWarning: X has feature names, but RidgeClassifierCV was fitted without feature names
  warnings.warn(


0.8669354838709677